In [1]:
import pandas as pd
import numpy as np
import random

# --- VETERINARY TRIAGE DATA GENERATOR ---
# CITATION SOURCES:
# 1. AVECCS (Assoc. of Vet Emergency & Critical Care Specialists) Triage List
# 2. Modified Manchester Triage System (MTS) for Small Animals
# 3. Ettinger & Feldman: Textbook of Veterinary Internal Medicine (Vital Ranges)

def generate_clinical_data(num_samples=5000):
    data = []
    
    for _ in range(num_samples):
        # --- 1. PHYSIOLOGICAL VARIABLES (Based on Canine Normals) ---
        # Normal Temp: 38.3 - 39.2 C. 
        # We generate a bell curve but allow for outliers (sick dogs).
        temp = round(np.random.normal(38.6, 0.8), 1) 
        
        # Heart Rate: Normal 60-140 bpm. 
        # Sick dogs often have Tachycardia (>160) or Bradycardia (<60).
        heart_rate = int(np.random.normal(100, 25)) 
        
        # --- 2. CLINICAL SIGNS (Binary: 0=Absent, 1=Present) ---
        # Probabilities adjusted to reflect typical ER caseloads
        vomiting = np.random.choice([0, 1], p=[0.75, 0.25])
        diarrhea = np.random.choice([0, 1], p=[0.75, 0.25])
        lethargy = np.random.choice([0, 1], p=[0.6, 0.4])
        
        # CRITICAL SIGNS (Rare but distinct)
        pain_vocalization = np.random.choice([0, 1], p=[0.9, 0.1]) 
        pale_gums = np.random.choice([0, 1], p=[0.95, 0.05]) # Indicator of Shock
        seizure = np.random.choice([0, 1], p=[0.98, 0.02])
        abdominal_distension = np.random.choice([0, 1], p=[0.97, 0.03]) # Indicator of GDV/Bloat
        
        # --- 3. TRIAGE LOGIC (The "Ground Truth" Labeling) ---
        # 0 = NON-URGENT (Green): Can wait hours/days.
        # 1 = URGENT (Yellow): Needs attention within hours.
        # 2 = EMERGENT/CRITICAL (Red): Immediate intervention required.
        
        urgency = 0 # Default to Healthy
        
        # [CRITICAL / RED RULES - AVECCS Level 1]
        if seizure == 1:
            urgency = 2
        elif pale_gums == 1:
            urgency = 2 # Hypovolemic Shock
        elif abdominal_distension == 1:
            urgency = 2 # Suspected GDV (Bloat)
        elif temp > 40.5: 
            urgency = 2 # Heat Stroke risk
        elif temp < 37.0: 
            urgency = 2 # Hypothermia/Shock
        elif heart_rate > 200:
            urgency = 2 # Severe Tachycardia (Arrhythmia risk)
            
        # [URGENT / YELLOW RULES - AVECCS Level 2]
        elif urgency == 0: # Only check if not already Red
            if temp > 39.7:
                urgency = 1 # Pyrexia (Fever)
            elif vomiting == 1 and diarrhea == 1:
                urgency = 1 # Risk of Dehydration
            elif pain_vocalization == 1:
                urgency = 1 # Acute Pain
            elif heart_rate > 160: 
                urgency = 1 # Mild Tachycardia (Pain/Stress)
            elif lethargy == 1 and vomiting == 1:
                urgency = 1 # General Illness

        data.append([temp, heart_rate, vomiting, diarrhea, lethargy, 
                     pain_vocalization, pale_gums, seizure, abdominal_distension, urgency])

    # Convert to DataFrame
    cols = ['Temperature_C', 'Heart_Rate_BPM', 'Vomiting', 'Diarrhea', 'Lethargy', 
            'Pain_Vocalization', 'Pale_Gums', 'Seizure', 'Abdominal_Distension', 'Urgency_Level']
    
    df = pd.DataFrame(data, columns=cols)
    return df

# Generate and Save
df_triage = generate_clinical_data(5000)
df_triage.to_csv('validated_triage_data.csv', index=False)

print("✅ SUCCESS: Generated 5,000 Clinical Scenarios based on AVECCS Standards.")
print("\n--- DATA DISTRIBUTION (Should be mostly Healthy, some Sick) ---")
print(df_triage['Urgency_Level'].value_counts().sort_index())
print("\n0 = Non-Urgent, 1 = Urgent, 2 = Critical")

✅ SUCCESS: Generated 5,000 Clinical Scenarios based on AVECCS Standards.

--- DATA DISTRIBUTION (Should be mostly Healthy, some Sick) ---
Urgency_Level
0    3148
1    1241
2     611
Name: count, dtype: int64

0 = Non-Urgent, 1 = Urgent, 2 = Critical


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

# 1. Load the Validated Data
df = pd.read_csv('validated_triage_data.csv')

# 2. Separate Inputs (X) and Output (y)
X = df.drop('Urgency_Level', axis=1)
y = df['Urgency_Level']

# 3. Split Data (80% Training, 20% Testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Train the Model
# n_estimators=100: Creates 100 decision trees to ensure stability
model = RandomForestClassifier(n_estimators=100, random_state=42)
print("🧠 Training the AI Brain...")
model.fit(X_train, y_train)

# 5. Evaluate Performance
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"\n🎉 TRAINING COMPLETE!")
print(f"🎯 Accuracy: {accuracy * 100:.2f}%")
print("\n--- DETAILED REPORT ---")
print(classification_report(y_test, y_pred, target_names=['Green (Low)', 'Yellow (Med)', 'Red (High)']))

# 6. Sanity Check: Test a specific "Critical" case manually
# Scenario: Dog has High Fever (41.0) and Seizures (1)
test_case = [[41.0, 120, 0, 0, 1, 0, 0, 1, 0]] # Matches column order
prediction = model.predict(test_case)[0]
severity_map = {0: 'Green', 1: 'Yellow', 2: 'RED CRITICAL'}

print(f"\n🧪 TEST CASE (Fever + Seizure): AI Predicted -> {severity_map[prediction]}")

# 7. Save the Model
joblib.dump(model, 'vet_triage_model.pkl')
joblib.dump(X.columns.tolist(), 'model_columns.pkl')
print("💾 Model saved successfully.")

🧠 Training the AI Brain...

🎉 TRAINING COMPLETE!
🎯 Accuracy: 99.50%

--- DETAILED REPORT ---
              precision    recall  f1-score   support

 Green (Low)       1.00      1.00      1.00       626
Yellow (Med)       0.99      0.99      0.99       249
  Red (High)       1.00      0.98      0.99       125

    accuracy                           0.99      1000
   macro avg       0.99      0.99      0.99      1000
weighted avg       1.00      0.99      0.99      1000


🧪 TEST CASE (Fever + Seizure): AI Predicted -> RED CRITICAL
💾 Model saved successfully.


C:\Users\janak\AppData\Roaming\Python\Python314\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
